# Final Report on the Coursera Capstone Project

#### This notebook is abiut the final report on the capstone project. It will contain all the steps of the required exrcise

## Introduction/Business Problem